In [2]:
# !git clone https://github.com/baaivision/Painter

In [1]:
!nvidia-smi

Wed Jun 19 07:53:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|


|   0  NVIDIA RTX A6000               On  | 00000000:15:00.0 Off |                  Off |
| 77%   86C    P2             211W / 300W |   1911MiB / 49140MiB |     47%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000               On  | 00000000:2D:00.0 Off |                  Off |
| 76%   87C    P2             288W / 300W |  19039MiB / 49140MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   G

In [2]:
%cd '/storage/scratch1/e18-4yp-medical-img-anno/Painter/SegGPT/SegGPT_inference'

/storage/scratch1/e18-4yp-medical-img-anno/Painter/SegGPT/SegGPT_inference


In [3]:
# !wget https://huggingface.co/BAAI/SegGPT/resolve/main/seggpt_vit_large.pth

In [4]:
# !pip3 install -r requirements.txt

In [5]:
import torch
print(torch.__version__) #current version
print(torch.cuda.is_available()) #check if GPU is available

2.3.0
True


In [6]:
import os
import random
import shutil

In [7]:
# images_folder="/storage/scratch1/e18-4yp-medical-img-anno/tufts/tufts/images"
# images_folder="/storage/scratch1/e18-4yp-medical-img-anno/wbc_images"
# images_folder="/storage/scratch1/e18-4yp-medical-img-anno/FS_Segmentation/FSMS-Surrogate/ISIC2018/ISIC2018_Task1-2_Training_Input"
# images_folder="/storage/scratch1/e18-4yp-medical-img-anno/oasis_images"

In [8]:
# labels_folder="/storage/scratch1/e18-4yp-medical-img-anno/tufts/tufts/labels"
# labels_folder="/storage/scratch1/e18-4yp-medical-img-anno/isic_masks"
# labels_folder="/storage/scratch1/e18-4yp-medical-img-anno/wbc_masks"
# labels_folder="/storage/scratch1/e18-4yp-medical-img-anno/oasis_labels"

In [9]:
# image_filenames = sorted(os.listdir(images_folder) )
# mask_filenames = sorted( os.listdir(labels_folder) )

In [10]:
# (len(image_filenames),len(mask_filenames))

In [11]:
# combined_filenames = list(zip(image_filenames, mask_filenames))
# random.shuffle(combined_filenames)
# split_index = int(0.7 * len(combined_filenames))

# support_filenames = combined_filenames[:split_index]
# test_filenames = combined_filenames[split_index:]

In [12]:
# (len(support_filenames),len(test_filenames))

In [13]:
# test_filenames[:5]

In [14]:
support_images_folder='/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/support/images'
support_labels_folder='/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/support/labels'
test_images_folder='/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/test/images'
test_labels_folder='/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/test/labels'

In [15]:
# support_images, support_labels = zip(*[(os.path.join(images_folder, image_filename), os.path.join(labels_folder, mask_filename)) for image_filename, mask_filename in support_filenames])

In [16]:
# save_support_images='/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/support/images'
# save_support_labels='/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/support/labels'

In [17]:
# def copy_files(file_paths, destination_folder):
#     for file_path in file_paths:
#         if os.path.exists(file_path):
#             destination_path = os.path.join(destination_folder, os.path.basename(file_path))
#             shutil.copy(file_path, destination_path)
#             print(f"Copied {file_path} to {destination_path}")
#         else:
#             print(f"File {file_path} does not exist and cannot be copied")


In [18]:
# # Copy the support images
# print("Start Copying Images....")
# copy_files(support_images, save_support_images)
# print("Start Copying Labels.....")
# # Copy the support labels
# copy_files(support_labels, save_support_labels)

In [19]:
# test_images, test_labels = zip(*[(os.path.join(images_folder, image_filename), os.path.join(labels_folder, mask_filename)) for image_filename, mask_filename in test_filenames])

In [20]:
# save_test_images='/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/test/images'
# save_test_labels='/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/test/labels'

In [21]:
# # Copy the support images
# print("Start Copying Images....")
# copy_files(test_images, save_test_images)
# print("Start Copying Labels.....")
# # Copy the support labels
# copy_files(test_labels, save_test_labels)

In [22]:
support_images=[]
support_labels=[]
test_images=[]
test_labels=[]
def create_arrays(arr,folder_path):
    for file in sorted(os.listdir(folder_path)):
        arr.append(os.path.join(folder_path,file))

In [23]:
create_arrays(support_images,support_images_folder)
create_arrays(support_labels,support_labels_folder)
create_arrays(test_images,test_images_folder)
create_arrays(test_labels,test_labels_folder)

In [24]:
# len(support_images)

In [25]:
n_support=100
_support_images=support_images[:n_support]
_support_labels=support_labels[:n_support]

In [26]:
_support_images[:3],_support_labels[:3]

(['/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/support/images/OASIS_OAS1_0001_MR1.jpg',
  '/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/support/images/OASIS_OAS1_0003_MR1.jpg',
  '/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/support/images/OASIS_OAS1_0004_MR1.jpg'],
 ['/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/support/labels/OASIS_OAS1_0001_MR1.jpg',
  '/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/support/labels/OASIS_OAS1_0003_MR1.jpg',
  '/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/support/labels/OASIS_OAS1_0004_MR1.jpg'])

In [27]:
test_images[:3]

['/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/test/images/OASIS_OAS1_0002_MR1.jpg',
 '/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/test/images/OASIS_OAS1_0006_MR1.jpg',
 '/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/test/images/OASIS_OAS1_0007_MR1.jpg']

In [28]:
test_labels[:3]

['/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/test/labels/OASIS_OAS1_0002_MR1.jpg',
 '/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/test/labels/OASIS_OAS1_0006_MR1.jpg',
 '/storage/scratch1/e18-4yp-medical-img-anno/evaluation/oasis/test/labels/OASIS_OAS1_0007_MR1.jpg']

In [29]:
len(test_images),len(test_labels)

(125, 125)

In [29]:
# image,label= test_filenames[1]
# image,label=os.path.join(images_folder, image), os.path.join(labels_folder, label)
# image,label

In [45]:
# !pip3 install timm==0.4.12

In [46]:
#one-shot inference
#evrything in an image with a prompt
# !python seggpt_inference.py \
# --input_image examples/hmbb_2.jpg \
# --prompt_image examples/hmbb_1.jpg \
# --prompt_target examples/hmbb_1_target.png \
# --output_dir ./

In [40]:
import shutil
prompt_images=" ".join(_support_images[:1])
prompt_labels=" ".join(_support_labels[:1])

# for image,label in test_filenames[:30]:
#     image_,label_=os.path.join(images_folder, image), os.path.join(labels_folder, label)
#     # shutil.copy(label_,"/storage/scratch1/e18-4yp-medical-img-anno/Painter/SegGPT/SegGPT_inference/outputs/tufts/gt/"+label)
#     !python seggpt_inference.py --input_image $image_ \
#         --label_image $label_ \
#         --prompt_image $prompt_images \
#         --prompt_target $prompt_labels \
#         --output_dir outputs/isic/1

for image_ in test_images[:30]:
    label_=test_labels[test_images.index(image_)]
    # image_,label_=os.path.join(images_folder, image), os.path.join(labels_folder, label)
    # shutil.copy(label_,"/storage/scratch1/e18-4yp-medical-img-anno/Painter/SegGPT/SegGPT_inference/outputs/tufts/gt/"+os.path.basename(label_))
    !python seggpt_inference.py --input_image $image_ \
        --label_image $label_ \
        --prompt_image $prompt_images \
        --prompt_target $prompt_labels \
        --output_dir outputs/oasis/1


Model loaded.
Score: 0.42311891913414
Finished.
Model loaded.
Score: 0.4005585312843323
Finished.
Model loaded.
Score: 0.40533247590065
Finished.
Model loaded.
Score: 0.3779354989528656
Finished.
Model loaded.
Score: 0.38892513513565063
Finished.
Model loaded.
Score: 0.347460001707077
Finished.
Model loaded.
Score: 0.41652408242225647
Finished.
Model loaded.
Score: 0.4174667298793793
Finished.
Model loaded.
Score: 0.3833063542842865
Finished.
Model loaded.
Score: 0.39106959104537964
Finished.
Model loaded.
Score: 0.4137114882469177
Finished.
Model loaded.
Score: 0.41468489170074463
Finished.
Model loaded.
Score: 0.3951108455657959
Finished.
Model loaded.
Score: 0.40031489729881287
Finished.
Model loaded.
Score: 0.3900022506713867
Finished.
Model loaded.
Score: 0.38097283244132996
Finished.
Model loaded.
Score: 0.40083202719688416
Finished.
Model loaded.
Score: 0.4035980999469757
Finished.
Model loaded.
Score: 0.4075584411621094
Finished.
Model loaded.
Score: 0.37490370869636536
Finishe

In [41]:
import numpy as np
scores=[]
score_file = os.path.join('/storage/scratch1/e18-4yp-medical-img-anno/Painter/SegGPT/SegGPT_inference/outputs/oasis/1', 'scores.txt')
with open(score_file, 'r') as f:
    for line in f.readlines():
        score = float(line.strip())
        scores.append(score)

np.array(scores).mean()
    

0.39488611817359925